# 1. Introducción

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [2]:
pd.set_option('display.max_columns', 1000) #ver todas las columnas

In [3]:
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

# 2. Preprocesamiento de los Datos

### Se quita id, titulo, descripción, direccion e Idzona por no entenderlos como tan relevantes

In [4]:
dfinicial = train
dftest = test

dfinicial = dfinicial.drop( columns = ['id', 'titulo', 'descripcion', 'direccion'] )
dftest = dftest.drop( columns = ['id', 'titulo', 'descripcion', 'direccion'] )

### Se quita latitud y longitud por la cantidad de NaN, manejando la granularidad sólo a nivel Ciudad y Provincia

In [5]:
dfinicial = dfinicial.drop( columns = ['lat', 'lng'] )
dftest = dftest.drop( columns = ['lat', 'lng'] )

### Se rellenan con la moda Baños, IdZona, Garages y Habitaciones

In [6]:
def completarAlgunosNan2(df):
    for column in ["banos", "garages", "habitaciones", "idzona"]:
        df[column] = df.groupby(['provincia', 'tipodepropiedad'])[column].apply(lambda x: x.fillna(x.mode()))
        df[column] = df[column].fillna(df[column].mode()[0]) #Por si son todos NaN, y no hay moda
        
    return df
    
dfinicial = completarAlgunosNan2(dfinicial)
dftest = completarAlgunosNan2(dftest)

### Se rellena Antigüedad, Metros Cubiertos y Metros totales con la mediana para los casos de NaN, por tener estos atributos distribución log normal (aproximadamente)

In [7]:
def completarAlgunosNan(df):
    for column in ['antiguedad', 'metroscubiertos', 'metrostotales']:
        df[column] = df.groupby(['provincia', 'tipodepropiedad'])[column].apply(lambda x: x.fillna(x.median()))
        df[column] = df[column].fillna(df[column].median())

    return df

dfinicial = completarAlgunosNan(dfinicial)
dftest = completarAlgunosNan(dftest)
        

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\nanfunctions.py:1112: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


### Aplicar Onehot encoding para desprenderse de provincias, ciudades tipo de propiedad como tal

In [8]:
dfinicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 17 columns):
tipodepropiedad               239954 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    240000 non-null float64
habitaciones                  240000 non-null float64
garages                       240000 non-null float64
banos                         240000 non-null float64
metroscubiertos               240000 non-null float64
metrostotales                 240000 non-null float64
idzona                        240000 non-null float64
fecha                         240000 non-null object
gimnasio                      240000 non-null float64
usosmultiples                 240000 non-null float64
piscina                       240000 non-null float64
escuelascercanas              240000 non-null float64
centroscomercialescercanos    240000 non-null float64
precio                       

In [9]:
# Se hallan los casos posibles
provincias_posibles = train["provincia"].dropna().unique()
props_posibles = train["tipodepropiedad"].dropna().unique()
#ciudades_posibles = train["ciudad"].dropna().unique()

# Se pasa tipodepropiedad y provincia a valores categóricos
dfinicial["tipodepropiedad"] = pd.Categorical(train["tipodepropiedad"], categories = props_posibles)
dfinicial["provincia"] = pd.Categorical(train["provincia"], categories = provincias_posibles)
#dfinicial["ciudad"] = pd.Categorical(train["ciudad"], categories = ciudades_posibles)

dftest["tipodepropiedad"] = pd.Categorical(test["tipodepropiedad"], categories=props_posibles)
dftest["provincia"] = pd.Categorical(test["provincia"], categories=provincias_posibles)
#dftest["ciudad"] = pd.Categorical(test["ciudad"], categories = ciudades_posibles)

# Rellenar nans de provincia, tipo de propiedad y ciudad, y usar onehot encoding

# COMENTAR EL QUE NO SE USE DE ESTOS 2
datos_categoricos = dfinicial[["tipodepropiedad", "provincia", "ciudad"]].copy()
datos_categoricos = dfinicial[["tipodepropiedad", "provincia"]].copy()

datos_categoricos['tipodepropiedad'] = datos_categoricos.groupby(['provincia'])['tipodepropiedad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos["provincia"] = datos_categoricos["provincia"].fillna(datos_categoricos["provincia"].mode()[0])
#datos_categoricos["ciudad"] = datos_categoricos.groupby(['provincia'])['ciudad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos = pd.get_dummies(datos_categoricos, sparse = True)
dfinicial = pd.concat([datos_categoricos, dfinicial], axis=1)

# COMENTAR EL QUE NO SE USE DE ESTOS 2
#datos_categoricos = dftest[["tipodepropiedad", "provincia", "ciudad"]].copy()
datos_categoricos = dftest[["tipodepropiedad", "provincia"]].copy()

datos_categoricos['tipodepropiedad'] = datos_categoricos.groupby(['provincia'])['tipodepropiedad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos["provincia"] = datos_categoricos["provincia"].fillna(datos_categoricos["provincia"].mode()[0])
#datos_categoricos["ciudad"] = datos_categoricos.groupby(['provincia'])['ciudad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos = pd.get_dummies(datos_categoricos, sparse = True)
dftest = pd.concat([datos_categoricos, dftest], axis=1)

# Tirar tipodepropiedad y provincia
dfinicial = dfinicial.drop( columns = ['tipodepropiedad', 'provincia', 'ciudad'] )
dftest = dftest.drop( columns = ['tipodepropiedad', 'provincia', 'ciudad'] )

### Se pasa fecha a datetime y se agrega mes y año

In [10]:
dfinicial['fecha'] = dfinicial['fecha'].astype('datetime64')
dftest['fecha'] = dftest['fecha'].astype('datetime64')

dfinicial['dia'] = dfinicial['fecha'].dt.day
dfinicial['mes'] = dfinicial['fecha'].dt.month
dfinicial['anio'] = dfinicial['fecha'].dt.year
dfinicial = dfinicial.drop( columns = ['fecha'] )

dftest['dia'] = dftest['fecha'].dt.day
dftest['mes'] = dftest['fecha'].dt.month
dftest['anio'] = dftest['fecha'].dt.year
dftest = dftest.drop( columns = ['fecha'] )

### Adecuación de tipos de datos ahora que no hay NaN

In [11]:
 def parseoNan(df):
    df['gimnasio'] = df['gimnasio'].astype(np.uint8)
    df['usosmultiples'] = df['usosmultiples'].astype(np.uint8)
    df['piscina'] = df['piscina'].astype(np.uint8)
    df['escuelascercanas'] = df['escuelascercanas'].astype(np.uint8)
    df['centroscomercialescercanos'] = df['centroscomercialescercanos'].astype(np.uint8)
    
    df['garages'] = df['garages'].astype(np.uint8)
    df['antiguedad'] = df['antiguedad'].astype(np.uint8)
    df['banos'] = df['banos'].astype(np.uint8)
    df['habitaciones'] = df['habitaciones'].astype(np.uint8)
    
    df['metroscubiertos'] = df['metroscubiertos'].astype(np.uint16)
    df['metrostotales'] = df['metrostotales'].astype(np.uint16)
   
    df['dia'] = df['dia'].astype(np.uint8)
    df['mes'] = df['mes'].astype(np.uint8)
    df['anio'] = df['anio'].astype(np.uint16)
    df['anio'] = df['anio'].astype(np.uint16)
    df['idzona'] = df['idzona'].astype(np.uint32)
    
parseoNan(dfinicial)
parseoNan(dftest)

dfinicial['precio'] = dfinicial['precio'].astype(np.uint32)

In [12]:
'''### Se quitan columnas correspondientes a ciudades que sólo están asociadas a una o dos propiedades#

cant_ciudad = pd.DataFrame( train.groupby("ciudad").size(), columns = ["cantidad"] ).reset_index()
ciudad_pocas = cant_ciudad[ cant_ciudad["cantidad"] < 3 ]

ciudades = [ "ciudad_" + p for p in ciudad_pocas['ciudad'].to_list() ]

dfinicial = dfinicial.drop( columns = ciudades )

dftest = dftest.drop( columns = ciudades )'''

'### Se quitan columnas correspondientes a ciudades que sólo están asociadas a una o dos propiedades#\n\ncant_ciudad = pd.DataFrame( train.groupby("ciudad").size(), columns = ["cantidad"] ).reset_index()\nciudad_pocas = cant_ciudad[ cant_ciudad["cantidad"] < 3 ]\n\nciudades = [ "ciudad_" + p for p in ciudad_pocas[\'ciudad\'].to_list() ]\n\ndfinicial = dfinicial.drop( columns = ciudades )\n\ndftest = dftest.drop( columns = ciudades )'

### Quitar día

In [13]:
dfinicial = dfinicial.drop(["dia"], axis=1)
dftest = dftest.drop(["dia"], axis=1)

# 3. Feature Engenieering

### Se agrega información sobre la población

In [14]:
poblacion_train = pd.read_csv(f"data/features/ftr_poblacion_train.csv")
poblacion_test = pd.read_csv(f"data/features/ftr_poblacion_test.csv")
dfinicial["poblacion"] = poblacion_train
dftest["poblacion"] = poblacion_test

### Se agrega información a partir de los strings del datatset

In [15]:
tiene_terraza = pd.read_csv(f"data/features/tiene_terraza_train.csv")
dfinicial["tiene_terraza"] = tiene_terraza
tiene_terraza_tes = pd.read_csv(f"data/features/tiene_terraza_test.csv")
dftest["tiene_terraza"] = tiene_terraza_tes

tiene_jardin = pd.read_csv(f"data/features/tiene_jardin_train.csv")
dfinicial["tiene_jardin"] = tiene_jardin
tiene_jardin_tes = pd.read_csv(f"data/features/tiene_jardin_test.csv")
dftest["tiene_jardin"] = tiene_jardin_tes

tiene_servicio = pd.read_csv(f"data/features/tiene_servicio_train.csv")
dfinicial["tiene_servicio"] = tiene_servicio
tiene_servicio_tes = pd.read_csv(f"data/features/tiene_servicio_test.csv")
dftest["tiene_servicio"] = tiene_servicio_tes

tiene_vestidor = pd.read_csv(f"data/features/tiene_vestidor_train.csv")
dfinicial["tiene_vestidor"] = tiene_vestidor
tiene_vestidor_tes = pd.read_csv(f"data/features/tiene_vestidor_test.csv")
dftest["tiene_vestidor"] = tiene_vestidor_tes

tiene_salon = pd.read_csv(f"data/features/tiene_salon_train.csv")
dfinicial["tiene_salon"] = tiene_salon
tiene_salon_tes = pd.read_csv(f"data/features/tiene_salon_test.csv")
dftest["tiene_salon"] = tiene_salon_tes

tiene_cuarto = pd.read_csv(f"data/features/tiene_cuarto_train.csv")
dfinicial["tiene_cuarto"] = tiene_cuarto
tiene_cuarto_tes = pd.read_csv(f"data/features/tiene_cuarto_test.csv")
dftest["tiene_cuarto"] = tiene_cuarto_tes

tiene_family = pd.read_csv(f"data/features/tiene_family_train.csv")
dfinicial["tiene_family"] = tiene_family
tiene_family_tes = pd.read_csv(f"data/features/tiene_family_test.csv")
dftest["tiene_family"] = tiene_family_tes

tiene_bosque = pd.read_csv(f"data/features/tiene_bosque_dir_train.csv")
dfinicial["bosque_dir"] = tiene_bosque
tiene_bosque_tes = pd.read_csv(f"data/features/tiene_bosque_dir_test.csv")
dftest["bosque_dir"] = tiene_bosque_tes

tiene_avenida = pd.read_csv(f"data/features/tiene_avenida_dir_train.csv")
dfinicial["avenida_dir"] = tiene_avenida
tiene_avenida_tes = pd.read_csv(f"data/features/tiene_avenida_dir_test.csv")
dftest["avenida_dir"] = tiene_avenida_tes

tiene_fuente = pd.read_csv(f"data/features/tiene_fuente_dir_train.csv")
dfinicial["fuente_dir"] = tiene_fuente
tiene_fuente_tes = pd.read_csv(f"data/features/tiene_fuente_dir_test.csv")
dftest["fuente_dir"] = tiene_fuente_tes

tiene_golf = pd.read_csv(f"data/features/tiene_golf_dir_train.csv")
dfinicial["golf_dir"] = tiene_golf
tiene_golf_tes = pd.read_csv(f"data/features/tiene_golf_dir_test.csv")
dftest["golf_dir"] = tiene_golf_tes

tiene_hectarea = pd.read_csv(f"data/features/tiene_hectarea_train.csv")
dfinicial["tiene_hectarea"] = tiene_hectarea
tiene_hectarea_tes = pd.read_csv(f"data/features/tiene_hectarea_test.csv")
dftest["tiene_hectarea"] = tiene_hectarea_tes

tiene_torre = pd.read_csv(f"data/features/tiene_torre_train.csv")
dfinicial["tiene_torre"] = tiene_torre
tiene_torre_tes = pd.read_csv(f"data/features/tiene_torre_test.csv")
dftest["tiene_torre"] = tiene_torre_tes

### Se usa THT con las descripciones y los títulos

In [16]:
n_dimensiones = 1000
desc_train = pd.read_csv(f"data/features/descr_vectorizadas_train_{n_dimensiones}d.csv")
desc_test = pd.read_csv(f"data/features/descr_vectorizadas_test_{n_dimensiones}d.csv")

In [17]:
tit_train = pd.read_csv(f"data/features/titulos_vectorizados_train_50d.csv")
tit_test = pd.read_csv(f"data/features/titulos_vectorizados_test_50d.csv")

dfinicial = pd.concat([dfinicial, desc_train, tit_train], axis=1)
dftest = pd.concat([dftest, desc_test, tit_test], axis=1)

### TF IDF

In [18]:
score_tf_train = pd.read_csv(f"data/features/scores_tfidf_train.csv")
dfinicial["score_tfidf"] = score_tf_train

score_tf_test = pd.read_csv(f"data/features/scores_tfidf_test.csv")
dftest["score_tfidf"] = score_tf_test

### Pasar a disco

In [20]:
dfinicial.shape

(240000, 1136)

In [21]:
dftest.shape

(60000, 1135)

In [19]:
detenerse

NameError: name 'detenerse' is not defined

In [ ]:
dftest.to_csv(f"data/test_final.csv", index=False)
dfinicial.to_csv(f"data/train_final.csv", index=False)